In [ ]:
import os
import sys
import yaml
import optuna
import os.path as osp
import pytorch_lightning as pl

from pathlib import Path
from argparse import ArgumentParser, Namespace
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TestTubeLogger, TensorBoardLogger

from datamodule import DataModule
from supervised import SupervisedLightningModule

import sys
sys.path.append('../')
from utils import run_cli, validate_path, yaml_func

/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def run_cli(config_path='configs/config.yaml'):
    validate_path(config_path)
    with open(config_path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

In [ ]:
config_path='configs/config.yaml'
validate_path(config_path)
with open(config_path, 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
config = run_cli()

In [ ]:
import torch
torch.load(config['hparams']['model_path_fine'])

{'epoch': 86,
 'global_step': 16856,
 'pytorch-lightning_version': '1.4.1',
 'state_dict': OrderedDict([('model.conv1.0.weight',
               tensor([[[[ 0.0257,  0.0218, -0.1250],
                         [-0.0875, -0.1554, -0.1417],
                         [ 0.1683,  0.1429,  0.0457]],
               
                        [[ 0.1420, -0.0711, -0.1034],
                         [ 0.1260, -0.0213,  0.0190],
                         [ 0.0849,  0.0750, -0.1029]],
               
                        [[-0.0702, -0.1593, -0.0021],
                         [-0.0638,  0.0249,  0.0002],
                         [ 0.1319,  0.0096,  0.0748]]],
               
               
                       [[[-0.0077,  0.0526, -0.0716],
                         [ 0.0487,  0.0486, -0.0660],
                         [ 0.0394,  0.0724, -0.0820]],
               
                        [[-0.0587,  0.0330,  0.0502],
                         [-0.0068,  0.0152,  0.0483],
                         [ 0.0

In [ ]:
def main() -> None:
    config = run_cli()
    seeds = config['seeds']
    tk=0.1
    step=0.1
    k=config['hparams']['k']
    while k<1:
        config['hparams']['k']=tk
        tk=tk+step
    
    
        for seed in seeds:
            if seed is not None:
                pl.seed_everything(seed)


            ckpt_callback = ModelCheckpoint(
                filename='{epoch}-{val_loss:.2f}',
                **config['ckpt_callback'],
            )
            if 'callbacks' in config['trainer_params']:
                config['trainer_params']['callbacks'] = yaml_func(
                    config['trainer_params']['callbacks'])
            if config['trainer_params']['default_root_dir'] == "None":
                config['trainer_params']['default_root_dir'] = osp.dirname(__file__)
        
            lr = config['hparams']['lr']
        
            config['logger']['version'] =f'layer=only1|lr={lr}|'
        
            model = SupervisedLightningModule(config)

            logger = TensorBoardLogger(
                config['logger']['save_dir'],
                name=config['logger']['name'] + f"-seed{seed}",
                version=config['logger']['version'],)

            trainer = pl.Trainer(**config['trainer_params'],
                                callbacks=[ckpt_callback],
                                logger=logger)
            imdm = DataModule(
                train_dir=config['dataset']['train_dir'],
                val_dir=config['dataset']['test_dir'],
                raw_data_dir=config['dataset']['raw_data_dir'],
                batch_size=config['hparams']['batch_size'],
                num_workers=config['hparams']['num_workers'],
                split=False,
                split_file=None)
            trainer.fit(model, datamodule=imdm)


if __name__ == '__main__':
    main()

Global seed set to 4
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1292: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet9          | 6.6 M 
1 | ce_loss | CrossEntropyLoss | 0     
---------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.527    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Global seed set to 4


Epoch 0:   1%|▏         | 3/236 [00:09<08:53,  2.29s/it, loss=10.2, v_num=001|]

/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Global seed set to 4
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/yanfeng/miniconda3/envs/bilateral/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1292: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet9          | 6.6 M 
1 | ce_loss | CrossEntropyLoss | 0     
---------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.527    Total estimated model params size (MB)


Global seed set to 4


Epoch 0:   1%|          | 2/236 [00:05<06:49,  1.75s/it, loss=10.6, v_num=001|]

from argparse import Namespace
from typing import Optional, Dict, Any
import sys
sys.path.append('../')

import torch
import torchvision.utils as vutils
import torch.nn.functional as F
from torch import Tensor, nn
from torch.optim.optimizer import Optimizer

from models.resnet import unicameral, resnet18, resnet34
from models.resnet_v0 import bicameral, resnet9
from pytorch_lightning import LightningModule
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR

from sklearn.metrics import accuracy_score
from utils import plot_grad_flow, plot_grad_flow_v2, \
        inverse_normalize, matplotlib_imshow

In [ ]:
from argparse import Namespace
from typing import Optional, Dict, Any
import sys
sys.path.append('../')

import torch
import torchvision.utils as vutils
import torch.nn.functional as F
from torch import Tensor, nn
from torch.optim.optimizer import Optimizer

from models.resnet import unicameral, resnet18, resnet34
from models.resnet_v0 import bicameral, resnet9
from pytorch_lightning import LightningModule
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR

from sklearn.metrics import accuracy_score
from utils import plot_grad_flow, plot_grad_flow_v2, \
        inverse_normalize, matplotlib_imshow

In [ ]:
mydict={ 
            "carch": "resnet9",
            "farch": "invresnet9", 
            "mode": "feature",
            "cmodel_path": "/home/chandramouli/kaggle/cerenaut/classification/logs/left-right-broad-sparsity-0.9-linear/res9_512-fc20|lr=1.0e-4|/checkpoints/epoch=97-val_acc=0.77.ckpt",
            "fmodel_path": "/home/chandramouli/kaggle/cerenaut/classification/logs/narrow-pyramid-sparsity-seed0/lr=0.0001|opt=adam|/checkpoints/epoch=99-val_acc=0.61.ckpt",
            "cfreeze_params": True,
            "ffreeze_params": True,
            }

In [ ]:
args = Namespace(**mydict)
print(args)

In [ ]:
bicameral(args)

In [ ]:
def _initialize_model(self):
        """
        """
        mydict={ 
            "carch": "resnet9",
            "farch": "invresnet9", 
            "mode": "feature",
            "cmodel_path": "/home/chandramouli/kaggle/cerenaut/classification/logs/left-right-broad-sparsity-0.9-linear/res9_512-fc20|lr=1.0e-4|/checkpoints/epoch=97-val_acc=0.77.ckpt",
            "fmodel_path": "/home/chandramouli/kaggle/cerenaut/classification/logs/narrow-pyramid-sparsity-seed0/lr=0.0001|opt=adam|/checkpoints/epoch=99-val_acc=0.61.ckpt",
            "cfreeze_params": True,
            "ffreeze_params": True,
            }
        args = Namespace(**mydict)
        self.model = bicameral(args)